In [13]:
# reading DARDAR .h5 files and outputting monthly files for totals 
#  and counts onto a chosen grid, which can then be read by another
#  python script to create a multi-month mean grid
# dduncan, 14/12/17
# updated 2/1/18 for doing day and night separately (or both in future, i guess)
# 25/1/18 taking the makegrid_dardar python script and adapting to 2C-ICE,
#  didn't get unzip to work on 2c hdf files so unzipped on dendrite. code for
#  integrating profiles from IWC itself should be okay or close (didnt realize there
#  was an actual IWP field in the data)

#import h5py
from mpl_toolkits.basemap import Basemap, cm
from pyhdf.SD import SD, SDC
import pyhdf.VS
from pyhdf.HDF import *

#from zipfile import ZipFile  as zp #zipfile
import zipfile
import shutil
import tempfile
import os

from isa import isa

In [14]:
#def open_zhdf(fname):
#    if fname.endswith(".zip"):
#        infile = zipfile.ZipFile.open(fname,mode='r')
#        tmp = tempfile.NamedTemporaryFile(delete=False)
#        shutil.copyfileobj(infile, tmp)
#        infile.close()
#        tmp.close()
#        handle = HDF(tmp.name).vstart()# = netCDF4.Dataset(tmp.name)
#        os.unlink(tmp.name)
#    else:
#        handle = HDF(fname).vstart()
#    return handle

In [15]:
#def open_zhdfdata(eff):  # for the 'data' fields, not the 'geo' fields, ugh
#    if eff.endswith(".zip"):
#        print(eff)
#        infile2 = zipfile.ZipFile.open( eff, mode='r')
#        tmp2 = tempfile.NamedTemporaryFile(delete=False)
#        shutil.copyfileobj(infile2, tmp2)
#        infile2.close()
#        tmp2.close()
#        handle2 = SD(tmp2.name,SDC.READ) 
#        os.unlink(tmp2.name)
#    else:
#        handle2 = SD(fname2,SDC.READ) 
#    return handle2

In [16]:

dir = '/home/dudavid/Dendrite/Dendrite/SatData/CloudSat/2C-ICE.R04/'
yrs = ['08']#7']#,'08'] # day and night data years on dendrite
dn = 'all'
startlat = 60.0 # abs value of NS boundaries
gsize = 2.5 # grid size in degree
nx, ny = int(360/gsize), int(startlat*2/gsize)
#nz = 125

#plvs = [1030,1000,975,950,925,900,875,850,825,800,775,750,700,650,600,550,\
#        500,450,400,350,300,250,200,150,100] #shrink by a couple to p levels common to ERA/MERRA
#plvs = plvs[::-1]
#nnz= len(plvs)-1
#print(plvs)

# try doing 6 pixel long average of DARDAR data
nsmo = 3 # double this so 6? -- [ x-nsmo : n+nsmo ]
#iwpall = []

In [25]:
for yr in yrs:
    #yr = '09'
    print('starting year 20'+yr)
    dirt = dir+'20'+yr
    months = sorted(os.listdir(dirt))
    iwpg,iwpgc = np.zeros([ny,nx]), np.zeros([ny,nx]) # re-init below after writing out
    days = sorted(os.listdir(dirt))
    ndays = 0 #init
    iwpall = []
    for d in days[ndays:]:
        print('day ',d)
        files = sorted(os.listdir(dirt+'/'+d))
        if len(files) != 0:  # in case no files in day dir
            ndays += 1
            for fix in files: #range(len(files)):
                file = dirt+'/'+d+'/'+fix
                #print(file)
                #f = open_zhdfdata(file) #SD(file,SDC.READ)
                #f = SD(file,SDC.READ)
                #iwc = f.select('IWC')[:,:] # works because data field!
                #print(np.shape(iwc),type(iwc))
                #hgt  = f.select('Height')[:]  # [pix, height]
                #gee = open_zhdf(file)
                gee = HDF(file).vstart()
                #print(gee.vdatainfo())
                lat = np.array(gee.attach('Latitude')[:])
                lon = np.array(gee.attach('Longitude')[:])
                iwp = np.array(gee.attach('ice_water_path')[:])
                #print(info(iwp),np.shape(iwp))
                mask = [abs(lat) < startlat]
                lat = lat[mask]
                lon = lon[mask]
                iwp = iwp[mask]
                n = np.size(lat)

                # there isn't a day/night flag in 2c-ice data, so would need to devise a separator 
                #iwc = iw[mask,:] # just selected profiles
                #lat = lat[mask]
                #lon = lon[mask]
                
                #zees = range(nz)
                #iwc_temp = np.zeros([n,nnz]) +np.nan
                #for l in range(n):
                #    dsubs = np.zeros([nz]) -1
                #    ct = 0
                #    for z in range(nz)[0:102]:
                #        if hgt[l,z]>500 and hgt[l,z]<16000:  
                #            ptemp = isa(hgt[l,z])  # input alt, get pressure
                #            ptempp= isa(hgt[l,z+1])  # input alt, get pressure
                #            if ptemp < 100*plvs[ct+1] and ptemp >= 100*plvs[ct]: dsubs[z] = ct
                #            if ptempp > 100*plvs[ct+1] and ptempp < 100*plvs[ct+2]: ct+=1
                #    if np.max(dsubs) > 0:
                #        for ez in range(nnz):
                #            if len(dsubs==ez) > 0:
                #                iwc_temp[l,ez] = np.mean(iwc[l,dsubs==ez]) # take avg of all D obs within EC
                    
                #iwp = np.array([np.nansum(iwc_temp[x,:]*250.0) for x in range(n)])
                #print(info(iwp))
                #goodz = [iwp==iwp]
                ariwp = np.array(iwp[:])#,0])#[goodz]) #[iwp>0])
                n = np.size(ariwp) # in case trimmed for say day/night only
                #print('N is ',n)

                    # running mean over iwp series:
                iwpsmoo = ariwp # first few will be same, so will last few. 
                iwpsmoo[nsmo:(n-nsmo)] = \
                    [ariwp[(x-nsmo):(x+nsmo)].mean() for x in range(nsmo,(n-nsmo))]

                iwpall.extend(0.001*iwpsmoo[::nsmo]) # no point storing all points...
                # convert to kg to match other histo datasets
                
                # calculate lat/lon indices for Xdeg grid (fixed 23/11):
                ladex = np.round((lat[:]+startlat-gsize*.5) / gsize)
                lodex = np.round((lon[:]+180.0-gsize*.5) / gsize)

                for x in range(nx):
                    if np.any(lodex == x):
                        dexsub = np.where(lodex == x)[0]
                        subset = iwpsmoo[dexsub] # using smoothed iwp or not?
                        subsetla = ladex[dexsub]
                        for y in range(ny):
                            if np.any(subsetla == y):
                                lolasubset = np.where(subsetla == y)[0]
                                iwpg[y,x] += subset[lolasubset].sum() # add up, get mean afterward
                                iwpgc[y,x] += len(subset[lolasubset])


        if ndays % 10 == 0 or d == '365':
            nd = str(ndays).rjust(3,'0')
            # dataset name, res (x,y), time res of input, output arrays (grids, zonals), version?, # days
            fin = 'grids/ice2c.'+dn+'.'+str(nx)+'x.'+str(ny)+'y.iwp.v1.'+yr+'.'+nd
            #fin = 'grids/dardar.'+str(nx)+'x.'+str(ny)+'y.iwptot.cts.hist.v1.'+yr+m+'.'+nd
            np.save(fin+'.tot',iwpg)
            np.save(fin+'.cts',iwpgc)
            print('wrote out '+nd)
            iwpg,iwpgc = np.zeros([ny,nx]), np.zeros([ny,nx]) # re-init after writing out

        #nd = str(ndays).rjust(2,'0')
        # create histogram of IWP
            binz = [2**x for x in range(-14,6)]
            binzz = [-.001,2**(-14)]
            #print(np.shape(iwpall),type(iwpall))
            #print(binz,binzz)
            ariwpa = np.array(iwpall)
            ariwpa = ariwpa[np.where(ariwpa==ariwpa)] #avoid NANs!
            #print(info(np.array(iwpall),-1))
            #hista, bin_edges  = np.histogram(np.array(iwpall),bins=binz)
            #histaz, bin_edges = np.histogram(np.array(iwpall),bins=binzz)
            hista, bin_edges  = np.histogram(ariwpa,bins=binz)
            histaz, bin_edges = np.histogram(ariwpa,bins=binzz)
            print('histaz',histaz,hista)
            fin = 'grids/ice2c.histo.60NS.all.v2.'+yr+'.'+nd
            outz = np.append(histaz,hista)
            np.save(fin,outz) #hista)
            iwpall = [] # reinit for next go round

starting year 2008
day  001
day  002
day  003
day  004
day  005
day  006
day  007
day  008
day  009
day  010
wrote out 010
histaz [500093] [ 6653 14155 25719 36109 41539 43468 42349 46152 49524 46497 40069 34205
 28245 23099 17310 10910  4438  1301     3]
day  011
day  012
day  013
day  014
day  015
day  017
day  018
day  022
day  023
day  024
wrote out 020
histaz [354988] [ 4655 10565 18989 27989 31781 30681 30059 32902 34609 32607 27943 22767
 19431 15470 11454  7658  3673   934    10]
day  025
day  026
day  027
day  028
day  029
day  030
day  031
day  032
day  033
day  034
wrote out 030
histaz [512891] [ 6705 14759 26953 37582 45231 44827 40983 44121 47228 45535 39361 32761
 26822 21192 16440 10450  4996  1363     3]
day  035
day  036
day  037
day  038
day  039
day  040
day  041
day  042
day  043
day  044
wrote out 040
histaz [615560] [ 7693 15629 28134 40651 47598 49342 47029 52872 56064 53385 46543 37802
 30835 24990 20344 13453  6133  1751    14]
day  045
day  046
day  047
day  0